In [2]:
from PIL import Image, ImageDraw, ImageFont  
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy import interpolate
import cv2 
from scipy.integrate import quad
from scipy.misc import derivative
import sympy as sp
import math
from math import sqrt, pi
# from scipy import integrate
from sympy import integrate, Symbol 
import scipy.integrate as spi
import Augmentor as Aug
import os
import shutil
import time
import import_ipynb # pip install import-ipynb # https://stackoverflow.com/questions/20186344/ipynb-import-another-ipynb-file
# import Operations # file operation.ipynb # https://github.com/mdbloice/Augmentor/tree/master/Augmentor
# import Pipeline

In [2]:
# Parameters
font_type = 'C:/Windows/Fonts/Calibri/calibrib.ttf'
font_size = 49 # 49
character = 'A'
threshold_binarisation = 100

In [3]:
# Befehl muss zweimal ausgeführt werden (komisch)
%matplotlib qt
%matplotlib qt

In [4]:
plt.close('all')
size_y, size_x = 200, 200
img = Image.new('RGB', (size_y, size_x), color = (0, 0, 0))   # (50,50)  
fnt = ImageFont.truetype(font_type, font_size)
d = ImageDraw.Draw(img)
d.text((20,20), character, font=fnt, fill=(255, 255, 255))
# img.show() 
# img.save('pil_text_font.bmp')
img = np.array(img)
plt.figure('img')
plt.imshow(img)
x,y,z = np.shape(img)
# print(x,y,z)
img[img<threshold_binarisation] = 0
plt.figure('binarised')
plt.imshow(img)
print(type(img))

x, y, w, h = cv2.boundingRect(img[:,:,0])
print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h

y_position = int((size_y-h)/2)
x_position = int((size_x-w)/2)
print(y_position,x_position)

img_centered = Image.new('RGB', (size_y, size_x), color = (0, 0, 0))   # (50,50)  
fnt = ImageFont.truetype(font_type, font_size)
d_centered = ImageDraw.Draw(img_centered)
d_centered.text((x_position,y_position-5), character, font=fnt, fill=(255, 255, 255))
# img.show() 
# img.save('pil_text_font.bmp')
img_centered = np.array(img_centered)
plt.figure('img_centered')
plt.imshow(img_centered)

img_centered_binarised = deepcopy(img_centered)
img_centered_binarised[img_centered_binarised<threshold_binarisation] = 0
img_centered_binarised[img_centered_binarised>=threshold_binarisation] = 255

plt.figure('img_center_binarised')
plt.imshow(img_centered_binarised)
print(type(img_centered_binarised))

   
img_inverted = np.abs(img_centered_binarised-255)*255
plt.figure('img_inverted')
plt.imshow(img_inverted)

img_size_uint8 = Image.fromarray(np.uint8(img_inverted))
img_size_uint8.save('C:/Users/User/Desktop/test/generateImages/' + 'test.bmp')



<class 'numpy.ndarray'>
21 26 28 31
84 86
<class 'numpy.ndarray'>


In [5]:
def image_rect(img):
    img_rect = deepcopy(img)
    x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
    # print(x, y, w, h)
    x_min = x-1
    y_min = y-1
    x_max = x+w
    y_max = y+h
    cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
    img_without_boundary = img_rect[y:y_max, x:x_max, :]
    return img_without_boundary

In [6]:
# img.astype(np.float32)
# img.astype(np.uint8)
# get the bounding rect
img_rect = deepcopy(img)
x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
# print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h
cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
plt.figure('img_rect')
plt.imshow(img_rect/255)

In [7]:
img_without_boundary = img_rect[y:y_max, x:x_max,:]
plt.figure('img_without_boundary')
plt.imshow(img_without_boundary/255)

In [8]:
# print(img_without_boundary)
y_img_without_boundary, x_img_without_boundary, z_img_without_boundary = np.shape(img_without_boundary)
# print(y_img_without_boundary, x_img_without_boundary, z_img_without_boundary)

In [9]:
tmp_x_min = int(-1*x_img_without_boundary/2)
tmp_x_max = int(np.ceil(x_img_without_boundary/2))
tmp_y_min = int(-1*y_img_without_boundary/2)
tmp_y_max = int(np.ceil(y_img_without_boundary/2))
# print(tmp_x_min)
# print(tmp_x_max)
# print(tmp_y_min)
# print(tmp_y_max)
x_mesh = np.arange(tmp_x_min, tmp_x_max, 1)
y_mesh = np.arange(tmp_y_min, tmp_y_max, 1)
# print(x_mesh)
# print(y_mesh)
xx, yy = np.meshgrid(x_mesh, np.flip(y_mesh,0), sparse=True)
yy = yy*1j
z = (yy+xx)
# print(z)
q = np.pi/180
c_meshgrid = 0.5*z.imag*1j + 2*z.real  # (z*np.cos(q)-np.sin(q)) / (z*np.sin(q)+np.cos(q*5))
# print(c_meshgrid)
y_meshgrid, x_meshgrid = c_meshgrid.shape
# print(y_meshgrid, x_meshgrid)
# c_meshgrid_reshape = c_meshgrid.reshape((y_meshgrid,x_meshgrid,1))

In [10]:
c_meshgrid_mask = deepcopy(c_meshgrid)
max_real_meshgrid_mask = int(np.max(c_meshgrid_mask.real))
max_imag_meshgrid_mask = int(np.max(c_meshgrid_mask.imag))
min_real_meshgrid_mask = int(np.min(c_meshgrid_mask.real))
min_imag_meshgrid_mask = int(np.min(c_meshgrid_mask.imag))
# print(max_real_meshgrid_mask)
# print(max_imag_meshgrid_mask)
# print(min_real_meshgrid_mask)
# print(min_imag_meshgrid_mask)
# print(c_meshgrid_mask.shape)

In [11]:
z_transform = np.array(np.zeros((max_imag_meshgrid_mask+1-min_imag_meshgrid_mask, max_real_meshgrid_mask-min_real_meshgrid_mask+1, 3)))
y_z_transform, x_z_transform, z_z_transform = z_transform.shape
# print(z_transform)
# print(y_z_transform, x_z_transform, z_z_transform)

In [12]:
for i in range(y_meshgrid):
    for j in range(x_meshgrid):
        for k in range(3):
            z_transform[int(max_imag_meshgrid_mask-c_meshgrid_mask[i,j].imag), int(c_meshgrid_mask[i,j].real-min_real_meshgrid_mask),:] = img_without_boundary[i,j,k]
        
#   Für Y-Achse => max-y
#   Für X-Achse => x-min

In [13]:
print(z_transform.shape)
print(img_without_boundary.shape)

(15, 55, 3)
(31, 28, 3)


In [14]:
plt.figure('transformation')
plt.imshow(z_transform/255)

# Without complex numbers <br>

In [ ]:
plt.figure('img_without_boundary')
plt.imshow(img_without_boundary/255)
y_img, x_img, z_img = img_without_boundary.shape
print(y_img, x_img, z_img)

In [ ]:
# setting the x - coordinates 
x_plot = np.arange(-1*x_img/2, x_img/2 , 0.1) 
# print(x_plot)
# setting the corresponding y - coordinates 
# potting the points 
f_plot = x_plot
plt.figure('plot')
plt.plot(x_plot, f_plot)
plt.grid(True)
# function to show the plot 
plt.show() 

In [ ]:
import scipy.integrate as integrate
x = sp.Symbol('x')  
f = x**2
# shift half pixel to the right 
f_derviate = sp.diff(f) 
print(type(f_derviate)) 
# func = sp.Symbol(str(f_derviate))
func = f_derviate
print(func)
print(type(func))

u = np.zeros((10, 3), dtype=sp.Symbol)

def get_function(x):
    return x

tmp = integrate.quad(lambda x: sp.Symbol('f_derviate'),0,1)
print(type(tmp))

In [ ]:
# https://www.youtube.com/watch?v=QAqadzwo19Q
# a = quad(lambda x: sqrt(x**2+1),0,5)
# print(type(x))
# print(a)
# print(type(float(a[0])))

In [ ]:
index = np.array(np.zeros((1,40)))
for i in range(0,math.ceil(x_img/2)):
    function = lambda x: d
    tmp = quad(get_function(x),0,2)
    
    
#     index[0,i] = round(float(tmp[0]))
#     print(index[0,i])
#     if index[0,i] >= math.ceil(x_img/2):
#         break

#     print(index[0,i])
#     print(type(index[0,0]))

print(tmp[0])

In [ ]:
max_value = i-1 
print(max_value)
print(x_img/2)

In [ ]:
empty_field = np.array(np.zeros((y_img, x_img, z_img)))
print(empty_field.shape)
for i in range(0,y_img):
    for j in range(0,2*max_value):
        for z in range(0,z_img):
            l = math.floor(j-max_value)
#             print(l)
            tmp = quad(lambda x: sqrt(x**2+1),0,l)
            tmp = round(tmp[0])
            print(tmp)
#             print(j)
            if img_without_boundary[i,j,z] > 0:
                    empty_field[i,tmp+max_value,z] = img_without_boundary[i,j,z]
plt.figure('after_transforamtion')
plt.imshow(empty_field/255)

In [ ]:
b = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
b[1:7:2]
np.array([1, 3, 5])
b[-2:10]
np.array([8, 9])
b[-3:3:-1]
np.array([7, 6, 5, 4])

In [ ]:
print(img_without_boundary[:,i,:])

In [ ]:
# https://www.math.ubc.ca/~pwalls/math-python/differentiation/#arc-length
def arc_length(f,a,b,h=0.001,N=1000):
    '''Approximate the ac length of y=f(x) from x=a to x=b.

    Parameters
    ----------
    f : (vectorized) function of one variable
    a,b : numbers defining the interval [a,b]
    h : step size to use in difference formulas
    N : number of subintervals in trapezoid method

    Returns
    -------
    Approximation of the integral \int_a^b \sqrt{1 + (f'(x))^2} dx
    representing the arc length of y=f(x) from x=a to x=b.
    '''
    x = np.linspace(a,b,N+1)
    y = f(x)

    # Compute central difference formula for x_k for 1 <= k <= N-1
    h = np.min([h,(b-a)/N]) # Make sure that h is smaller than the size of the subintervals
    x_interior = x[1:-1]
    df_interior = (f(x_interior + h) - f(x_interior - h))/(2*h)

    # Use forward/backward difference formula at the endpoints
    df_a = (f(a + h) - f(a))/h
    df_b = (f(b) - f(b - h))/h
    df = np.hstack([[df_a],df_interior,[df_b]])

    # Compute values of the integrand in arc length formula
    y = np.sqrt(1 + df**2)

    # Compute the integral
    L = spi.trapz(y,x)

    return L

In [ ]:
a = 1
result = arc_length(lambda x: a*x**2,0,1,h=10**(-10),N=10**5)
print(result)

In [ ]:
def function(x)
    return x**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Fixing random state for reproducibility
np.random.seed(123456789)
y_max = 50

fig, ax = plt.subplots()
line, = ax.plot(np.random.rand(y_max))
ax.set_ylim(0, 1)


def update(data):
    line.set_ydata(data)
    return line,


def data_gen():
    while True:
        yield np.random.rand(y_max)

ani = animation.FuncAnimation(fig, update, data_gen, interval=100)
plt.show()

# Transformation from internet modified <br>

In [34]:
path_folder = 'C:/Users/User/Desktop/test/generateImages/' # Bilder Pfad
path_folder, dirs, files = next(os.walk(path_folder))
number_images = len(files)
# save_folder = 'C:/Users/User/Desktop/test/Augmentor/GenerateImages/'
print(path_folder)
print(number_images)
print(files)

path_output = path_folder + 'output/' # Bilder Pfad
path_output, dirs, output_files = next(os.walk(path_output))
output_number_images = len(files)
print(path_output)
print(output_number_images)
print(output_files)

C:/Users/User/Desktop/test/generateImages/
1
['test.bmp']
C:/Users/User/Desktop/test/generateImages/output/
1
[]


In [41]:
for i in range(1000):
    skew_magnitude = np.round(np.random.uniform(0.1, 1.0),1) # sm 0.1-1.0
    rd_grid_with = np.random.randint(2,10) # rdgw 2-10
    rd_grid_hight = np.random.randint(2,10) # rdgh 2-10
    rd_magnitude = np.random.randint(1,10) # # rdm 1-10 ### 20 to much

    
    p = Aug.Pipeline(path_folder, output_directory='output', save_format="BMP")
    p.random_distortion(probability=1, grid_width=rd_grid_with, grid_height=rd_grid_hight, magnitude=rd_magnitude)
    p.rotate(probability=1, max_left_rotation=25, max_right_rotation=25) # >= 25° for botch
    p.skew(probability=0.5, magnitude=skew_magnitude)
    p.shear(probability=0.5, max_shear_left=25, max_shear_right=25) # >= 25° for botch
    p.sample(1) 
    path_output, dirs, output_files = next(os.walk(path_output))
    timestr = time.strftime('%Y%m%d_%H%M%S')
    name = str(timestr) + '_' + 'sm' + str(skew_magnitude) + '_' + 'rdgw' + str(rd_grid_with) + '_' + 'rdgh' + str(rd_grid_hight) + '_' + 'rdm' + str(rd_magnitude)
#     os.rename(path_output + output_files[0], path_output + name + '.BMP')
    shutil.move(path_output + output_files[0], path_output + 'renamed/' + name + '.BMP')
    time.sleep(0.1)
    
print(type(p))
# print(b.shape)
# img_size_uint8 = Image.fromarray(np.uint8(img_mini))
# img_size_uint8.save(save_folder +

Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages/output.

Processing <PIL.Image.Image image mode=RGB size=200x200 at 0x826FE48>: 100%|█| 1/1 [00:00<00:00, 23.81 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages/output.

Processing <PIL.Image.Image image mode=RGB size=200x200 at 0x826F588>: 100%|█| 1/1 [00:00<00:00, 71.42 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages/output.

...

Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages/output.

Processing <PIL.Image.Image image mode=RGB size=200x200 at 0x826FA20>: 100%|█| 1/1 [00:00<00:00, 76.92 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages/output.

Processing <PIL.Image.Image image mode=RGB size=200x200 at 0x826FF98>: 100%|█| 1/1 [00:00<00:00, 41.66 Samples/s]


<class 'Augmentor.Pipeline.Pipeline'>


In [42]:
path_renamed = path_output + 'renamed/' # Bilder Pfad
path_renamed, dirs, renamed_files = next(os.walk(path_renamed))
renamed_number_images = len(renamed_files)
print(path_renamed)
print(renamed_number_images)
# print(renamed_files)

C:/Users/User/Desktop/test/generateImages/output/renamed/
1000


In [43]:
threshold = 150
for i in range (renamed_number_images):
    with Image.open(path_renamed + renamed_files[i]) as file:
        img = np.array(file)
        y,x,z = img.shape
        img = img[3:y-4,3:x-4,:]
        img_binarised = deepcopy(img)
        img_binarised[img<threshold] = 0
        img_binarised[img>=threshold] = 255
        img_inverted = abs(img_binarised-255)*255
 
        
#         plt.figure('img')
#         plt.imshow(img)
        
        img_without_boundary = image_rect(img_inverted)
        
#         print(img_without_boundary.shape)
#         plt.figure('img_without_boundary')
#         plt.imshow(img_without_boundary)

        img_size_uint8 = Image.fromarray(np.uint8(img_without_boundary))
        img_size_uint8.save(path_output + 'without_boundary_binarised/' + renamed_files[i])

In [78]:
print(path_output)
    

0 0 280 280


In [30]:
# https://github.com/mdbloice/Augmentor
# https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
# https://www.ritchieng.com/machine-learning-photo-ocr/

In [31]:
# def iterate():
#     img_output = np.zeros((rows+512, cols+512))
#     for i in range(rows):
#         for j in range(cols):
#             offset_x = int(15.0 * np.sin(6*np.pi*i/180))
#             offset_y = 0 #int(d * np.sin(e * 3.14 * 1*(i+f)))
# #             print(offset_x,offset_y)
# #             print(i+offset_y+h,j+offset_x+d)

#             img_output[i+offset_y+200,j+offset_x+200] = img[i,j]
            
# #             plt.figure('Input')
# #             plt.imshow(img, cmap='gray')
# #             plt.figure('Vertical wavecols')
# #             plt.imshow(img_output, cmap='gray')

#     plt.figure('bla')
#     plt.imshow(img_output/255)

#     img_rect = deepcopy(img_output)
#     x, y, w, h = cv2.boundingRect(img_rect)
#     # print(x, y, w, h)
#     x_min = x-1
#     y_min = y-1
#     x_max = x+w
#     y_max = y+h
#     cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
#     plt.figure('img_rect')
#     plt.imshow(img_rect/255)


    
#     img_save = image_rect(img_output)
#     img_size_uint8 = Image.fromarray(np.uint8(img_output))
#     img_size_uint8.save(path + str(a)+'_'+str(b)+'_'+str(c)+'_'+str(d)+'.bmp')
                    
#         img_size_uint8 = Image.fromarray(np.uint8(img_size))
#         img_size_uint8.save(path_save_image + files[i])

In [ ]:
# https://subscription.packtpub.com/book/application_development/9781785283932/1/ch01lvl1sec16/image-warping
# img_2D = img[:,:,0]
a = 15
b = 3
c = 1
d = 1

rows, cols = img_2D.shape 
img_output = np.zeros((200, 200))
img_output_3D = np.zeros((200, 200,3))
print(img_output.shape)
print(img.shape)
for i in range(rows):
    for j in range(cols):
        offset_x = int(a * np.sin(2*np.pi*i*b/180))
        offset_y = 0 # int(c * np.sin(2*np.pi*j*d/180))
#       print(offset_x,offset_y)
#       print(i+offset_y+h,j+offset_x+d)
        img_output[i+offset_y+50, j+offset_x+50] = img_2D[i,j]

# plt.figure('img_output')
# plt.imshow(img_output)
# print(type(img_output))

img_output_3D[:,:,0] = img_output
img_output_3D[:,:,1] = img_output
img_output_3D[:,:,2] = img_output

img_rect = deepcopy(img_output_3D)
img_rect = img_rect.astype(np.uint8)
x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
# print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h
cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
# plt.figure('img_rect')
# plt.imshow(img_rect/255)

img_without_boundary = img_rect[y:y_max, x:x_max,:]
plt.figure('img_without_boundary'+str(a)+'_'+str(b)+'_'+str(c)+'_'+str(d))
plt.imshow(img_without_boundary/255)


In [ ]:
import os

In [300]:
class GenerateData():
    
    all_character = ('ABCDEFGHIJKLMNOPQRSTUVWXYabcdefghijklmnopqrstuvwxy0123456789')
    character_folder = []
    
    def __init__(self,path, create_folder_structur = False,  *character):
        self.__path = path + '/GeneratedData/'
        self.__character = character
        self.__create_folder_structur = create_folder_structur
        self.__initalImages_path = path + '/GeneratedData/InitalImages/'
        self.__tempFolder_path = path + '/GeneratedData/TempFolder/'
        self.__distortedImages_path = path + '/GeneratedData/DistortedImages/'
        self.__tailoredImages_path = path + '/GeneratedData/TailoredImages/'
        self.__backgroundImages_path = path + '/GeneratedData/BackgroundImages/'
        self.__segmentedImages_path = path + '/GeneratedData/SegmentedImages/'
        self.__finalImages_path = path + '/GeneratedData/FinalImages/'
        self.__xml_Files_path = path + '/GeneratedData/XML_Files/'
        
        
        if self.__create_folder_structur == True:
            self.set_folder_structur()
    
    def set_character_folder(self):
        
        for i in range(len(self.__character)):
            if len(self.__character[i]) == 1:
                type(self).character_folder.append(self.__character[i][0])
#                 print(self.__character[i][0])
            else:
                
                for j in range(type(self).all_character.find(self.__character[i][0]), type(self).all_character.find(self.__character[i][2])+1):
                    type(self).character_folder.append(type(self).all_character[j])
#                     print(type(self).all_character[j])
                
                                                          
        for k in range(len(type(self).character_folder)):
            os.mkdir(self.__initalImages_path + type(self).character_folder[k])                                              
            os.mkdir(self.__tempFolder_path + type(self).character_folder[k])
            os.mkdir(self.__distortedImages_path + type(self).character_folder[k])    
            os.mkdir(self.__tailoredImages_path + type(self).character_folder[k])
                                                          
    def set_folder_structur(self):
        os.mkdir(self.__path)
        os.mkdir(self.__initalImages_path)
        os.mkdir(self.__tempFolder_path)
        os.mkdir(self.__distortedImages_path)
        os.mkdir(self.__tailoredImages_path)
        os.mkdir(self.__backgroundImages_path)
        os.mkdir(self.__segmentedImages_path)
        os.mkdir(self.__finalImages_path)
        os.mkdir(self.__xml_Files_path)
        if len(self.__character) > 0:
            self.set_character_folder()
            
        
    

In [302]:
GenerateData('C:/Users/User/Desktop/test/', True, ('D-H'),('A'), ('C'), ('K-P'), ('0-9'))

In [301]:
shutil.rmtree('C:/Users/User/Desktop/test/GeneratedData') 

In [122]:
u = ('A-F','a-i')
len(u)
print(u[0][2])

F


In [94]:
h = []
h.append('5')

In [66]:
A_Z.find('D')

3

In [215]:
def test(*d):
    print(d[0][1])
    print(d)

In [216]:
test(('asdfasdf'))

s
('asdfasdf',)


In [248]:
all_character = ('ABCDEFGHIJKLMNOPQRSTUVWXYabcdefghijklmnopqrstuvwxy0123456789')
all_character.find('D')

3

In [3]:
path_folder = 'C:/Users/User/Desktop/test/generateImages/' # Bilder Pfad
path_folder, dirs, files = next(os.walk(path_folder))
number_images = len(files)
# save_folder = 'C:/Users/User/Desktop/test/Augmentor/GenerateImages/'
print(path_folder)
print(number_images)
print(files)

C:/Users/User/Desktop/test/generateImages/
1
['test.bmp']


In [6]:
import os.path  
import glob  
folder = glob.glob('C:/Users/User/Desktop/test/GeneratedData/')
len(folder)

1

In [11]:
import os
cpt = sum([len(files) for r, d, files in os.walk("C:/Users/User/Desktop/test/GeneratedData/")])
print(cpt)

0


In [12]:
print(os.walk("C:/Users/User/Desktop/test/GeneratedData/"))

<generator object walk at 0x00000000059D0BA0>


In [14]:
import os
from os.path import join, getsize
for root, dirs, files in os.walk('C:/Users/User/Desktop/test/GeneratedData/'):
#     print(root, "consumes", end=" ")
#     print(sum(getsize(join(root, name)) for name in files), end=" ")
#     print("bytes in", len(files), "non-directory files")

C:/Users/User/Desktop/test/GeneratedData/ consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/BackgroundImages consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\0_000026 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\1_000027 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\2_000028 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\3_000029 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\4_000030 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\5_000031 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/DistortedImages\6_000032 con

C:/Users/User/Desktop/test/GeneratedData/TempFolder\Q_000016 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\R_000017 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\S_000018 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\T_000019 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\U_000020 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\V_000021 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\W_000022 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\X_000023 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\Y_000024 consumes 0 bytes in 0 non-directory files
C:/Users/User/Desktop/test/GeneratedData/TempFolder\Z_000025 consumes 0 b

In [21]:
root, dirs, files = os.walk('C:/Users/User/Desktop/test/GeneratedData/')
print(len(files))

TypeError: object of type 'generator' has no len()

In [19]:
print(root)

<generator object walk at 0x000000000B8FCD00>


In [22]:
path_folder, dirs, files = os.walk('C:/Users/User/Desktop/test/GeneratedData/')

ValueError: too many values to unpack (expected 3)